Import necessary libraries and import the data

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

In [2]:
temp_o = pd.read_csv('Annual_Surface_Temperature_Change.csv')
dis_o = pd.read_csv('Climate-related_Disasters_Frequency.csv')
fc_o = pd.read_csv('Forest_and_Carbon.csv')
land_o = pd.read_csv('Land_Cover_Accounts.csv')
co_o = pd.read_csv('owid-co2-data.csv')
ccode = pd.read_csv('iso-country-codes.csv')

In [3]:
ccode.rename(columns={'Alpha-3 code': 'ISO3'}, inplace=True)

## Clean the data

There is a column for each year in the datasets, which needs to be transposed into the same column for analysis. Therefore, I used the melt function to do so. Note that each dataset has a different number of years, so the second argument within the function grabs the correct number of columns for that dataframe

In [4]:
def clean_dataset(df, number):
    columns = df.columns.tolist()
    column_list = columns[number:]
    melt_df = pd.melt(df, id_vars=['Country','Unit','Indicator','ISO3'], value_vars=column_list,
               var_name='Year', value_name='value')
    melt_df['Year'] = melt_df['Year'].str[1:]
    return melt_df

In [5]:
temp = clean_dataset(temp_o,-62)
dis = clean_dataset(dis_o,-43)
fc = clean_dataset(fc_o,-29)
land = clean_dataset(land_o,-29)

Disaster df had a lot of different values in it, so I wanted to create a df for each. Had to do some manipulation for it, including: 
1. grabbing the value out of the 'indicator' section (drought, flood, etc.)
2. remove a space
3. create a new df for each based on the indicator

In [6]:
dis.fillna(0,inplace=True)

In [7]:
dis['Indicator'] = dis['Indicator'].str.split(':').str[-1]
dis['Indicator'] = dis['Indicator'].str.replace(' ', '')

Pivot all the values of disasters into columns, then only grab the total disasters

In [8]:
dis = dis.pivot(index=['Country','Year','ISO3'], columns='Indicator', values='value').reset_index()
dis.fillna(0,inplace=True)

In [9]:
dis = dis[['Country', 'Year', 'ISO3', 'TOTAL']]
dis.rename(columns={'TOTAL': 'total_disasters'}, inplace=True)

Only select applicable temperature dataframe

In [10]:
temp.rename(columns={'value': 'temp_c'}, inplace=True)

In [11]:
temp = temp[['ISO3','Country','Year','temp_c']]

Pivot data for fc and land

In [12]:
fc_pivoted = fc.pivot_table(index=['Country','Year','ISO3'], columns='Indicator', values='value', aggfunc='mean').reset_index()

In [13]:
land_pivoted = land.pivot_table(index=['Country','Year','ISO3'], columns='Indicator', values='value', aggfunc='mean').reset_index()

Join all the columns based on their ISO3 value (since that's standardized)

In [14]:
land = pd.merge(ccode[['ISO3']], land_pivoted, on='ISO3', how='inner')
fc = pd.merge(ccode[['ISO3']], fc_pivoted, on='ISO3', how='inner')
temp = pd.merge(ccode[['ISO3']], temp, on='ISO3', how='inner')
dis = pd.merge(ccode[['ISO3']], dis, on='ISO3', how='inner')

In [15]:
land['Year'] = land['Year'].astype(int)
fc['Year'] = fc['Year'].astype(int)
temp['Year'] = temp['Year'].astype(int)
dis['Year'] = dis['Year'].astype(int)

Merge data into one dataframe (for future analysis, can use inputs to predict values of outputs

In [16]:
df_inputs = pd.merge(land, fc, on=['Country', 'Year', 'ISO3'])

In [17]:
df_outputs = pd.merge(temp, dis, on=['Country', 'Year', 'ISO3'])

In [18]:
df = pd.merge(df_outputs, df_inputs, on=['Country', 'Year', 'ISO3'])

Remove all countries that have at least one NaN value in the temp_c or total_disasters

In [19]:
ISO3_List = df.ISO3.unique()

In [20]:
NaN_values = df[pd.isna(df['temp_c']) | pd.isna(df['total_disasters'])]

In [21]:
NaN_list = NaN_values.ISO3.unique()

In [22]:
iso_list = []
for c in ISO3_List:
    if c not in NaN_list:
        iso_list.append(c)

In [23]:
df = df[df['ISO3'].isin(iso_list)]

Drop all columns that have NaN values for simplicity

In [24]:
df = df.dropna(axis=1)

### Normalize the Data
To compare the land features of countries, we need to normalize the data by making the first year's data a value of 1, then calculating percentage changes

In [25]:
df_normalized = df.copy()

Right now there is a dataframe with all countries in it, so we can't normalize each column. Therefore, I created a separate dataframe for each country to run the calculation, then combined them into a dataset again

In [26]:
split_dataframes = {}

In [27]:
grouped = df.groupby('ISO3')

In [28]:
append_df = []

In [29]:
for ISO3, group in grouped:
    for column in group.columns[5:]:
        if group[column].iloc[0] == 0:
            group[column] = group[column]
        else:
            group[column] = group[column] / group[column].iloc[0]
    locals()[ISO3] = group
    append_df.append(group)

In [30]:
normalized_df = pd.concat(append_df, ignore_index=True)

Analyze one to start, which can be repeated by changing the variable

### CO2 Data

In this database, there are many types of co2, so we want to find them from the database. co2 is the total column, so we want to verify that the total of the type_co2 add up to the total

In [31]:
US = co_o[co_o["iso_code"] == "USA"]

In [32]:
selected_columns = [
    "country",
    "year",
    "iso_code",
    "gdp",
    "population",
    "co2",
    "cement_co2",
    "coal_co2",
    "consumption_co2",
    "flaring_co2",
    "gas_co2",
    "oil_co2",
    "other_industry_co2"
]

In [33]:
U = US[selected_columns]

In [34]:
U['co2_total'] = U['cement_co2'] + U['coal_co2'] + U['flaring_co2'] + U['gas_co2'] + U['oil_co2'] + U['other_industry_co2']

/Users/rossurbina/.conda/envs/ENVIRONMENT/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


shown below, co2 = co2_total, so we determined all values of the co2 for our analysis

In [35]:
U.tail()

,country,year,iso_code,gdp,population,co2,cement_co2,coal_co2,consumption_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,co2_total
48154,United States,2017,USA,1.759628e+13,329791232.0,5210.958,40.324,1308.479,5635.312,59.245,1460.392,2314.675,27.842,5210.957
48155,United States,2018,USA,1.814065e+13,332140032.0,5376.657,38.971,1256.202,5804.678,72.787,1618.720,2365.283,24.694,5376.657
48156,United States,2019,USA,NaN,334319680.0,5259.144,40.896,1070.724,5692.483,88.383,1673.999,2358.831,26.311,5259.144
48157,United States,2020,USA,NaN,335942016.0,4715.691,40.688,878.973,5197.398,67.684,1653.678,2049.336,25.332,4715.691
48158,United States,2021,USA,NaN,336997632.0,5007.336,41.203,1002.300,NaN,67.684,1637.038,2233.778,25.332,5007.335


Gather only columns we care about

In [36]:
co = co_o.copy()

In [37]:
co = co[selected_columns]

Get the per capita number of co2. Make the population in millions

In [38]:
co['population'] = co['population'] / 1000000

Remove NA values that don't have an ISO code, because we only want to look at countries (vs. groups like North America)

In [39]:
co.dropna(subset=['iso_code'], inplace=True)

Melt the data to make one column with all the values for better time analysis

In [40]:
co_melt = pd.melt(co, id_vars =['country','year','iso_code','gdp','population'], value_vars =co.columns[5:],
              var_name ='Type_co2', value_name ='Total')

In [41]:
co_melt['Per_Capita'] = co_melt['Total'] / co_melt['population']

In [42]:
co_melt[(co_melt['iso_code']=='CHN') & (co_melt['Type_co2']=='co2')]

,country,year,iso_code,gdp,population,Type_co2,Total,Per_Capita
7922,China,1850,CHN,3.534960e+11,409.821554,co2,NaN,NaN
7923,China,1851,CHN,NaN,407.685621,co2,NaN,NaN
7924,China,1852,CHN,NaN,404.840117,co2,NaN,NaN
7925,China,1853,CHN,NaN,401.291154,co2,NaN,NaN
7926,China,1854,CHN,NaN,397.773301,co2,NaN,NaN
...,...,...,...,...,...,...,...,...
8089,China,2017,CHN,1.757508e+13,1410.275968,co2,10011.107,7.098687
8090,China,2018,CHN,1.815162e+13,1417.069440,co2,10353.877,7.306542
8091,China,2019,CHN,NaN,1421.864064,co2,10740.996,7.554165
8092,China,2020,CHN,NaN,1424.929792,co2,10956.213,7.688949


## Export data to csv for Tableau

##### Tableau Public only takes csv files, so need to export csv files then import those into Tableau Public, vs. a direct connect from Python

co_melt.to_csv("/Users/rossurbina/Box Sync/Data Analytics Learning/Data Projects/Climate_Data/Export/co2_levels.csv")

land.to_csv("/Users/rossurbina/Box Sync/Data Analytics Learning/Data Projects/Climate_Data/Export/land.csv")
fc.to_csv("/Users/rossurbina/Box Sync/Data Analytics Learning/Data Projects/Climate_Data/Export/fc.csv")

temp.to_csv("/Users/rossurbina/Box Sync/Data Analytics Learning/Data Projects/Climate_Data/Export/temp.csv")
dis.to_csv("/Users/rossurbina/Box Sync/Data Analytics Learning/Data Projects/Climate_Data/Export/dis.csv")

df.to_csv("/Users/rossurbina/Box Sync/Data Analytics Learning/Data Projects/Climate_Data/Export/climate_data.csv")

df_normalized.to_csv("/Users/rossurbina/Box Sync/Data Analytics Learning/Data Projects/Climate_Data/Export/climate_data_normalized.csv")